In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff

largeDataSet = './data/raw/KDDTrain.arff'
smallDataSet = './data/raw/KDDTrain_20Percent.arff'

largeTestData = './data/raw/KDDTest.arff'
smallTestData = './data/raw/KDDTest-21.arff'

# Loading the .arff data format into a pandas dataframe
data, meta = arff.loadarff(largeDataSet)
test_data, test_meta = arff.loadarff(largeTestData)

train = pd.DataFrame(data)
test = pd.DataFrame(test_data)

print('Shape of the training data: ', train.shape)
print('Shape of the test data: ', test.shape)

Shape of the training data:  (125973, 42)
Shape of the test data:  (22544, 42)


In [95]:
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()

# for feature in categorical_cols:
#     plt.figure(figsize=(17, 5)) 
#     sns.countplot(data=df, x=feature, palette='Set3')
#     plt.title(f'Distribution of {feature}')
#     plt.xticks(rotation=85)
#     plt.show()

In [96]:

print([col for col in train if train[col].nunique() == 1])
print([col for col in test if test[col].nunique() == 1])
print(train.shape , test.shape)

['num_outbound_cmds']
['num_outbound_cmds']
(125973, 42) (22544, 42)


In [97]:

train = train.drop(columns='num_outbound_cmds')
test = test.drop(columns='num_outbound_cmds')
print(train.shape , test.shape)

(125973, 41) (22544, 41)


In [98]:
from sklearn.preprocessing import OneHotEncoder

train_categorical = train[train.select_dtypes(include=['object']).columns]
test_categorical = test[test.select_dtypes(include=['object']).columns]

train_categorical = train_categorical.drop(columns='class')
test_categorical = test_categorical.drop(columns='class')

encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(train_categorical)

train_encoded = encoder.transform(train_categorical)
test_encoded = encoder.transform(test_categorical)

print(train_encoded.shape , test_encoded.shape)

(125973, 92) (22544, 92)


In [99]:
from sklearn.preprocessing import MinMaxScaler

train_numerical = train[train.select_dtypes(include=['float64']).columns]
test_numerical = test[test.select_dtypes(include=['float64']).columns]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_numerical)
test_scaled = scaler.transform(test_numerical)

print(train_scaled.shape , test_scaled.shape)

(125973, 33) (22544, 33)


In [100]:
# train_scaled = pd.DataFrame(train_scaled, columns=train_numerical.columns)
# test_scaled = pd.DataFrame(test_scaled, columns=test_numerical.columns)

# train_encoded = pd.DataFrame(train_encoded.todense())
# test_encoded = pd.DataFrame(test_encoded.todense())

# processed_train = pd.concat([train_scaled, train_encoded], axis=1)
# processed_test = pd.concat([test_scaled, test_encoded], axis=1)

# print(processed_train.shape, processed_test.shape)
# processed_train.info()


# Convert scaled arrays back to DataFrames with original column names
train_scaled = pd.DataFrame(train_scaled, columns=train_numerical.columns)
test_scaled = pd.DataFrame(test_scaled, columns=test_numerical.columns)

# Get the feature names for the encoded columns
encoded_columns = encoder.get_feature_names_out(input_features=train_categorical.columns)
train_encoded = pd.DataFrame(train_encoded.toarray(), columns=encoded_columns)
test_encoded = pd.DataFrame(test_encoded.toarray(), columns=encoded_columns)

# Concatenate the scaled and encoded DataFrames
processed_train = pd.concat([train_scaled, train_encoded], axis=1)
processed_test = pd.concat([test_scaled, test_encoded], axis=1)

print(processed_train.shape, processed_test.shape)



(125973, 125) (22544, 125)


In [101]:
# Calculate the correlation matrix for train dataset
corr_matrix_train = processed_train.corr(method='pearson')

print("Initial number of features in train dataset:", len(corr_matrix_train.columns))

# Calculate the correlation matrix for test dataset
corr_matrix_test = processed_test.corr(method='pearson')

print("Initial number of features in test dataset:", len(corr_matrix_test.columns))

# Define the correlation threshold
corr_threshold = 0.6

# Find highly correlated features for train dataset
high_corr_features_train = set()

# Iterate through upper triangle of the correlation matrix for train dataset
for i in range(len(corr_matrix_train.columns)):
    for j in range(i + 1, len(corr_matrix_train.columns)):
        if abs(corr_matrix_train.iloc[i, j]) > corr_threshold:
            high_corr_features_train.add(corr_matrix_train.columns[i])
            high_corr_features_train.add(corr_matrix_train.columns[j])

print("Number of features with high correlation in train dataset:", len(high_corr_features_train))

# Drop highly correlated features from both train and test datasets
processed_train = processed_train.drop(columns=high_corr_features_train)
processed_test = processed_test.drop(columns=high_corr_features_train)

print("Number of remaining features in train dataset:", len(processed_train.columns))
print("Number of remaining features in test dataset:", len(processed_test.columns))

print(processed_train.shape, processed_test.shape)

# plt.figure(figsize=(20, 20))
# sns.heatmap(processed_train.corr(method='pearson'), fmt='.1g', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', linewidths=3, linecolor='black')
# plt.show()


Initial number of features in train dataset: 125
Initial number of features in test dataset: 125
Number of features with high correlation in train dataset: 38
Number of remaining features in train dataset: 87
Number of remaining features in test dataset: 87
(125973, 87) (22544, 87)


In [102]:
X_train = processed_train.to_numpy()
y_train = train['class'].values

X_test = processed_test
y_test = test['class'].values

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


print(type(X_train))
print(type(y_train))


(125973, 87) (22544, 87)
(125973,) (22544,)
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [103]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)

ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.